In [61]:
import models
import pandas as pd

In [62]:
# METRIC = 'F1'
# ALGO = 'detection_correction'
# # ALGO = 'correction'
MODEL = 'CNN_256_filters_7_kernels'
COMMODITY = 'copper'
confidence_levels = [0.1, 0.2, 0.3, 0.45, 0.5, 0.55, 0.6, 0.7, 0.8, 0.9, 0.95]

In [63]:
# df = models.npy_to_bowpy(f'{COMMODITY}/test/predictions/test/{MODEL}_predictions.csv', f'copper/test/predictions/test_{METRIC}_{ALGO}/', confidence_levels)
df = models.npy_to_bowpy(f'{COMMODITY}/test/predictions/test/{MODEL}_predictions.csv', f'{COMMODITY}/test/predictions/', confidence_levels)
df

,pred,corr,true_positive,false_positive,test_F1_correction_rule_result_conf_0.1,test_F1_correction_rule_result_conf_0.2,test_F1_correction_rule_result_conf_0.3,test_F1_correction_rule_result_conf_0.45,test_F1_correction_rule_result_conf_0.55,test_F1_correction_rule_result_conf_0.5,...,test_Precision_detection_correction_rule_result_conf_0.2,test_Precision_detection_correction_rule_result_conf_0.3,test_Precision_detection_correction_rule_result_conf_0.45,test_Precision_detection_correction_rule_result_conf_0.55,test_Precision_detection_correction_rule_result_conf_0.5,test_Precision_detection_correction_rule_result_conf_0.6,test_Precision_detection_correction_rule_result_conf_0.7,test_Precision_detection_correction_rule_result_conf_0.8,test_Precision_detection_correction_rule_result_conf_0.95,test_Precision_detection_correction_rule_result_conf_0.9
0,1,0,0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,0,0,0,0
1,1,0,0,1,1,1,1,1,1,1,...,1,1,1,1,1,0,0,0,0,0
2,0,0,0,0,1,1,1,1,0,0,...,1,1,1,0,0,0,0,0,0,0
3,0,0,0,0,1,1,1,1,0,0,...,1,1,1,0,0,0,0,0,0,0
4,0,0,0,0,1,1,1,0,0,0,...,1,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,0,0,0,0,1,1,1,1,0,1,...,1,1,1,0,0,0,0,0,0,0
218,0,0,0,0,1,1,1,1,0,1,...,1,1,1,0,1,0,0,0,0,0
219,0,0,0,0,1,1,1,1,1,1,...,1,1,1,1,1,0,0,0,0,0
220,0,0,0,0,1,1,1,1,0,1,...,1,1,1,0,1,0,0,0,0,0


In [64]:
def calculate_metrics(df, rule_result_column):
    TP = ((df[rule_result_column] == 1) & (df['corr'] == 1)).sum()
    FP = ((df[rule_result_column] == 1) & (df['corr'] == 0)).sum()
    FN = ((df[rule_result_column] == 0) & (df['corr'] == 1)).sum()
    
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return f1, recall, precision

In [65]:
f1, recall, precision = calculate_metrics(df, "pred")
print(f'Base model results\nf1: {f1}\nrecall: {recall}\nprecision: {precision}')

Base model results
f1: 0.19672131147540986
recall: 0.17142857142857143
precision: 0.23076923076923078


In [66]:
df.to_numpy().dump('edcr/data/test.npy')